In [1]:
!pip install nemo_toolkit['all']

  Obtaining dependency information for nemo_toolkit[all] from https://files.pythonhosted.org/packages/59/31/58466536a75dc1eff0d38be4b008730526ae36dd9693304a2ecb78d6411f/nemo_toolkit-1.21.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 62.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 11.8 MB/s eta 0:00:00
  Obtaining dependency information for sphinx from https://files.pythonhosted.org/packages/b2/b6/8ed35256aa530a9d3da15d20bdc0ba888d5364441bb50a5a83ee7827affe/sphinx-7.2.6-py3-none-any.whl.metadata
  Obtaining dependency information for sphinxcontrib-bibtex from https://files.pythonhosted.org/packages/79/59/fafc5c480506cc356e2a7ea009d7c7d75812475b4385fe851ae55575661c/sphinxcontrib_bibtex-2.6.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 14.9 MB/s eta 0:

In [2]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm.notebook import tqdm
from glob import glob
import librosa
import warnings
import random
import torch
import gc
import torchaudio
import torchaudio.transforms as AT
import torch.nn as nn
import nemo.collections.asr as nemo_asr
from glob import glob
from tqdm import tqdm

warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.restore_from("/kaggle/input/nemo-speaker-recognition-tutorial/nemo_experiments/TitaNet-Finetune/titanet-large-finetune.nemo")

[NeMo W 2023-11-28 07:20:07 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /kaggle/working/data/train.json
    sample_rate: 16000
    labels:
    - 01XEzyHk3m
    - 02fltGFC4R
    - 02v9iLACH1
    - 03DNDfrrhU
    - 047AFYJhfo
    - 04cUq4DdGc
    - 05qumaqAnQ
    - 079QFXRpRI
    - 07ewy8crbV
    - 0A6QGl6pFi
    - 0AEqkdivFI
    - 0C50bZ6gpe
    - 0CAE3COBqO
    - 0D7i17DWsN
    - 0DX0T0fQ2x
    - 0F1sB1v3z3
    - 0FRodtzBMa
    - 0GTpSNAXPu
    - 0JKFvbmSEg
    - 0LVvsdnajH
    - 0MkECoR1sC
    - 0OJcXg7IiU
    - 0OTG0Ojs4N
    - 0OZ07S5Cmh
    - 0Tuq1VZiAr
    - 0U4XJzEuAQ
    - 0UiDAP583J
    - 0VdIAzbq8B
    - 0WgqZFpauz
    - 0YIwVoOL2R
    - 0Z1ro47wH4
    - 0ZLTzooDCM
    - 0cMlb9tyQB
    - 0dqMcU26ih
    - 0gzeYI3aon
    - 0iK53lVsLP
    - 0iP2yOIo0h
    - 0jLdDAdYnM
    - 0k7NngwujE
    - 0kNHXm1IEZ


[NeMo I 2023-11-28 07:20:07 features:289] PADDING: 16
[NeMo I 2023-11-28 07:20:12 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /kaggle/input/nemo-speaker-recognition-tutorial/nemo_experiments/TitaNet-Finetune/titanet-large-finetune.nemo.


In [4]:


train_path = glob("/kaggle/input/aiconnect-speaker-recognition/train/*/*.wav")
test_path = glob("/kaggle/input/aiconnect-speaker-recognition/test/*.wav")

print(len(train_path))
print(len(test_path))

37534
9231


In [5]:
train_df = pd.DataFrame(train_path, columns = ["wav_path"])
train_df["filename"] = train_df["wav_path"].str.split('/').str[-1]
train_df["label"] = train_df["wav_path"].str.split('/').str[-2]

test_df = pd.DataFrame(test_path, columns = ["wav_path"])
test_df["filename"] = test_df["wav_path"].str.split('/').str[-1]

submission_df = pd.read_csv("/kaggle/input/aiconnect-speaker-recognition/sample_submission.csv")
print(train_df.head())
print(test_df.head())
print(submission_df.head())

                                            wav_path           filename  \
0  /kaggle/input/aiconnect-speaker-recognition/tr...   Jqri9rPr48_4.wav   
1  /kaggle/input/aiconnect-speaker-recognition/tr...  Jqri9rPr48_11.wav   
2  /kaggle/input/aiconnect-speaker-recognition/tr...  Jqri9rPr48_10.wav   
3  /kaggle/input/aiconnect-speaker-recognition/tr...   Jqri9rPr48_6.wav   
4  /kaggle/input/aiconnect-speaker-recognition/tr...   Jqri9rPr48_2.wav   

        label  
0  Jqri9rPr48  
1  Jqri9rPr48  
2  Jqri9rPr48  
3  Jqri9rPr48  
4  Jqri9rPr48  
                                            wav_path         filename
0  /kaggle/input/aiconnect-speaker-recognition/te...  ZXJ0FVHIGvJ.wav
1  /kaggle/input/aiconnect-speaker-recognition/te...  Q3WUDALUvVy.wav
2  /kaggle/input/aiconnect-speaker-recognition/te...  ItsQ7DTw2nx.wav
3  /kaggle/input/aiconnect-speaker-recognition/te...  MDWmUSqwTcD.wav
4  /kaggle/input/aiconnect-speaker-recognition/te...  n4cGOAsWzm3.wav
         recording voice_id
0  00

In [6]:
tqdm.pandas()

train_df["embedding"] = train_df["wav_path"].progress_apply(lambda x : speaker_model.get_embedding(x))

100%|██████████| 37534/37534 [15:23<00:00, 40.62it/s]


In [7]:
train_df["embedding"] = train_df["embedding"].progress_apply(lambda x : x / torch.linalg.norm(x))

100%|██████████| 37534/37534 [00:00<00:00, 40679.30it/s]


In [8]:
train_df["embedding"][0]

tensor([[-0.0261, -0.0513, -0.1678,  0.0729,  0.0602,  0.0872,  0.1760,  0.0018,
          0.1183,  0.1473,  0.0449,  0.0990, -0.1366,  0.0400,  0.0867,  0.0597,
         -0.0869, -0.0664,  0.0271,  0.0453,  0.0272,  0.1006,  0.0207,  0.0241,
          0.0677,  0.0802, -0.0435, -0.0497,  0.1274,  0.1541, -0.0103,  0.0301,
          0.0433, -0.0779,  0.0054,  0.0624, -0.0559,  0.1335,  0.0290, -0.0442,
         -0.0235,  0.0007,  0.0336, -0.0322,  0.0637, -0.0276,  0.0591,  0.0890,
          0.0271, -0.0010, -0.0487, -0.0199, -0.0731, -0.1276, -0.0280, -0.1233,
          0.0630,  0.0744,  0.0793, -0.1769,  0.1251,  0.0786,  0.0505, -0.0660,
          0.0542,  0.0160, -0.0562,  0.0932,  0.0690,  0.1103,  0.0803, -0.0427,
         -0.0682,  0.0888,  0.0367, -0.0277, -0.0651, -0.1298,  0.0468, -0.0378,
         -0.0699,  0.0390,  0.1318,  0.0777,  0.0090,  0.0097, -0.0970,  0.0669,
         -0.0300, -0.0596, -0.0770,  0.0304,  0.0460,  0.0212,  0.0288, -0.0336,
          0.1388, -0.0438,  

In [9]:
test_df["embedding"] = test_df["wav_path"].progress_apply(lambda x : speaker_model.get_embedding(x))
test_df["embedding"] = test_df["embedding"].progress_apply(lambda x : x / torch.linalg.norm(x))

100%|██████████| 9231/9231 [00:00<00:00, 42841.03it/s]


In [10]:
train_group_df = train_df.groupby("label")["embedding"].sum()

In [11]:
train_group_df = train_group_df.progress_apply(lambda x : x / torch.linalg.norm(x))

100%|██████████| 1592/1592 [00:00<00:00, 42273.37it/s]


In [12]:
train_group_df.head()

label
01XEzyHk3m    [[tensor(-0.0662, device='cuda:0'), tensor(-0....
02fltGFC4R    [[tensor(0.0211, device='cuda:0'), tensor(0.06...
02v9iLACH1    [[tensor(0.0182, device='cuda:0'), tensor(0.04...
03DNDfrrhU    [[tensor(-0.0140, device='cuda:0'), tensor(0.0...
047AFYJhfo    [[tensor(-0.0668, device='cuda:0'), tensor(0.0...
Name: embedding, dtype: object

In [13]:
test_df = test_df.sort_values(by = ["filename"])
test_df = test_df.set_index("filename")

In [14]:
voice_ids = train_group_df.index

def get_similarity(embs1, embs2):
    embs1 = embs1.squeeze()
    embs2 = embs2.squeeze()
    X = embs1 / torch.linalg.norm(embs1)
    Y = embs2 / torch.linalg.norm(embs2)
    # Score
    similarity_score = torch.dot(X, Y) / ((torch.dot(X, X) * torch.dot(Y, Y)) ** 0.5)
    similarity_score = (similarity_score + 1) / 2
    return similarity_score


def get_voice_id(filename, threshold = 0.7):
    max_score = 0.0
    voice_id = "unknown"
    embs = test_df.loc[filename]["embedding"]
    for index, embs2 in train_group_df.items():
        sim = get_similarity(embs, embs2)
        if max_score < sim:
            voice_id = index
            max_score = sim
    if max_score < threshold:
        voice_id = "unknown"
    return voice_id

get_voice_id("ZXJ0FVHIGvJ.wav")

'U5njmTx1kg'

In [15]:
submission_df["voice_id"] = submission_df["recording"].progress_apply(get_voice_id)

100%|██████████| 9231/9231 [42:30<00:00,  3.62it/s]


In [16]:
print(submission_df["voice_id"])

0       QZjD7R9UXG
1       0D7i17DWsN
2       VYzPiTw3Kd
3       18Ft0jMO8z
4       UAjMk5dXJM
           ...    
9226    VGL2n0qwqj
9227    L6rB4MESeu
9228    ADHAbhp9WS
9229    QFxt4OFVD4
9230    V6BsRm1HtA
Name: voice_id, Length: 9231, dtype: object


In [17]:
submission_df.to_csv("submission.csv", index=False)